# Travaux Pratiques TP 2
-------------

# AES Execution on STM32F303 and Power Traces Re-Alignment

# TO DO
------------
We'll be collecting traces of our usual implementation of AES in this lab, with one small modification. If we use the EXTRA_OPTS=ADD_JITTER, a for loop that runs between 0 and 15 times through will be inserted:

```C
  #ifdef ADD_JITTER
  for (volatile uint8_t k = 0; k < (*pt & 0x0F); k++);
  #endif
```

As you can see, this jitter is actually based on the first byte of our plaintext. This in itself is a vulnerability. We won't use this fact, but as an exercise, try hypothocizing some ways to use this to overcome the jitter.

------------

---

---

First you'll need to select which hardware setup you have. 

You'll need to select a `SCOPETYPE`, a `PLATFORM`, and a `CRYPTO_TARGET`. 
* `SCOPETYPE` is `'OPENADC'` for the CWLite
* `PLATFORM` is the target device, with `'CWLITEARM'`
* `CRYPTO_TARGET` selects the crypto implementation, with `'TINYAES128C'` working on all platforms. 

For example:

```python
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='TINYAES128C'
SS_VER='SS_VER_1_1'
```

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='TINYAES128C'
SS_VER='SS_VER_1_1'

The following code will build the firmware for the target.

<img src='images/DoIt.png' style='width: 100px'>

## INDICATE BELOW IN YOUR NOTEBOOK THE RIGHT PATH TO `Setup_Scripts/Setup_Generic.ipynb` on YOUR MACHINE

In [ ]:
#%run ".../chipwhisperer/jupyter/Setup_Scripts/Setup_Generic.ipynb"

<img src='images/DoIt.png' style='width: 100px'>

## SAME: INDICATE BELOW IN YOUR NOTEBOOK THE RIGHT PATH TO `hardware/victims/firmware/simpleserial-aes` on YOUR MACHINE

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
#cd ../chipwhisperer/hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 EXTRA_OPTS=ADD_JITTER SS_VER=$3

<img src='images/DoIt.png' style='width: 100px'>

## SAME: INDICATE BELOW IN YOUR NOTEBOOK THE RIGHT PATH TO `hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex` on YOUR MACHINE

In [ ]:
#cw.program_target(scope, prog, "../chipwhisperer/hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

---------
## Run an AES computation and collect the power trace

In [ ]:
import numpy as np
import time

In [ ]:
ktp = cw.ktp.Basic()

In [ ]:
# Generate plaintext and key for the AES execution
key, plaintext = ktp.next()

# Set the AES key for the target
target.set_key(key)

# Send the plaintext to encrypt
target.simpleserial_write('p', plaintext)

# Define the number of samples to collect into the trace
scope.adc.samples = 5_000

# Arm the scope for trace acquisition
scope.arm()

# Run the AES
ret = scope.capture()
if ret:
    print("Target timed out!")

#Store the result
ciphertext_returned = np.array(target.simpleserial_read('r', 16))

#Store the trace collected
trace = np.array(scope.get_last_trace())

#### Print the plaintext used

#### Print the ciphertext returned by the board

### Print the points of the trace

* What is the trace length? how many points?



<img src='images/DoIt.png' style='width: 100px'>

### Verify the AES computation is correct

<img src='images/DoIt.png' style='width: 100px'>

### Plot the AES trace captured

<img src='images/DoIt.png' style='width: 100px'>


## Observe the trace 

Open the trace in full screen, zoom and observe

* What do you see?
* What kind of information do you visualize?


## We can use Holoviews

In [ ]:
import holoviews as hv
hv.extension('bokeh')

<img src='images/DoIt.png' style='width: 100px'>

### Try to zoom, dezoom, move the trace ...

<img src='images/DoIt.png' style='width: 100px'>


## Play with parameters:
* `scope.adc.samples` to increase or decrease the number of samples collected
* What do you observe about the computation?


In [ ]:
key, plaintext = ktp.next()
target.set_key(key)
# Send the plaintext to encrypt
target.simpleserial_write('p', plaintext)
# Define the number of samples to collect into the trace
scope.adc.samples = 24_000

scope.arm()
ret = scope.capture()
if ret:
    print("Target timed out!")

ciphertext_returned = np.array(target.simpleserial_read('r', 16))
trace = np.array(scope.get_last_trace())

### Plot the trace

<img src='images/DoIt.png' style='width: 100px'>


## What can you conclude?

<img src='images/DoIt.png' style='width: 100px'>


## What can you say about the AES?

In [ ]:
from matplotlib import colors as mcolors


colors = dict(mcolors.BASE_COLORS)
by_hsv = sorted((tuple(mcolors.rgb_to_hsv(mcolors.to_rgba(color)[:3])), name)
                for name, color in colors.items())
sorted_names = [name for hsv, name in by_hsv]

In [ ]:
period = 40
plt.rcParams['figure.figsize']=(16,4)
plt.title('AES Rounds', fontsize=14)    
#trace_obs = scared.signal_processing.moving_mean(trace, period).T
trace_obs = trace
beg_first_round = 550
round_length = 2680
borders = np.arange(beg_first_round, 25000, round_length)

for i in range(8):    plt.axvspan(borders[i-1], borders[i], color=sorted_names[i%5], alpha=0.15, label='toto')

plt.plot(trace)
plt.show()

---------

<img src='images/DoIt.png' style='width: 100px'>

## Run several AES executions and collect the corresponding power traces
## Traces Set Collection

In [ ]:
...

<img src='images/DoIt.png' style='width: 100px'>


### Plot 10 traces and their first 1000 points

## WHAT DO YOU OBSERVE HERE ?

<img src='images/DoIt.png' style='width: 100px'>

-----------
## What kind of Signal Processing Can we do with several traces? Objective : improve the trace observation !

### Compute the mean and the standard deviation on the set of collected traces

In [ ]:
plt.rcParams['figure.figsize']=(16,8)
trace_mean = np.mean(trace_array[0:100], axis = 0)
trace_std = np.std(trace_array[0:100], axis = 0)
plt.subplot(2,1,1) 
plt.plot(trace_mean.T)
plt.subplot(2,1,2) 
plt.plot(trace_std.T)
plt.show()

## Now we have collected:
TRACES
* a set of 100 power traces of AES executions
METADATA
* the corresponding 100 plaintext for each AES execution
* the corresponding 100 ciphertext computed by each AES execution
* the corresponding 100 key for each AES execution (can be the same or different)

### TRACES

In [ ]:
#%matplotlib
%matplotlib inline
plt.rcParams['figure.figsize']=(16,4)
plt.title('Trace AES collectée')
plt.plot(trace_array[0])
plt.plot(trace_array[1])
plt.plot(trace_array[2])
plt.show()

### METADATA

In [ ]:
textin_array[0:3]

In [ ]:
keyin_array[:3]

In [ ]:
response_array[:3]

In [ ]:
vhex = np.vectorize(hex)

In [ ]:
plaintext_np_array = np.array(textin_array)
plaintext_np_array.shape

In [ ]:
ciphertext_np_array = np.array(response_array)
ciphertext_np_array.shape

In [ ]:
key_np_array = np.array(keyin_array)
key_np_array.shape

In [ ]:
traces_np_array = np.array(trace_array)
traces_np_array.shape

<img src='images/DoIt.png' style='width: 100px'>

---------
## Using an open source SCA tool to analyse the trace with side-channel reverse engineering

In [ ]:
import estraces

In [ ]:
ths = ...

Plot ths[0] trace

In [ ]:
...

----
<img src='images/DoIt.png' style='width: 100px'>

## Reverse on intermediate values - First round

In [ ]:
import scared

In [ ]:
def ...

In [ ]:
container = scared.Container(...)

In [ ]:
Selection_function = ...

In [ ]:
Reverse_SB = ...
Reverse_SB.run(...)

In [ ]:
Reverse_SB.results.shape

### Plot results

<img src='images/DoIt.png' style='width: 100px'>

## What is the issue here? 


<img src='images/DoIt.png' style='width: 100px'>

------------
## Traces Resynchronization

We observe the traces are only shifted, so a first basic resynchronisation by shifting the trace should be enough

https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html#scipy.signal.find_peaks


In [ ]:
...

In [ ]:
...

In [ ]:
..

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(trace_resync_list[0])
ax.plot(trace_resync_list[1])
ax.plot(trace_resync_list[2])
ax.plot(trace_resync_list[3])
ax.plot(trace_resync_list[4])

ax.set_xlabel('Time samples', fontsize=15)
ax.set_ylabel('Trace amplitude', fontsize=15)
plt.plot()
plt.show()

In [ ]:
traces_resynchronised = np.array(trace_resync_list)

<img src='images/DoIt.png' style='width: 100px'>

## Now Redo the side-channel reverse analysis

In [ ]:
ths = estraces.read_ths_from_ram(samples=..., **{'plaintext': plaintext_np_array, 'ciphertext': ciphertext_np_array, 'key' : key_np_array})
print(ths)

In [ ]:
container = scared.Container(...)

In [ ]:
Selection_function = ...

In [ ]:
Reverse_SB = ...
Reverse_SB.run(...)

In [ ]:
Reverse_SB.results.shape

### Plot results